## COVID-19

Date: 2020, 31 Mar

Seminar for the graduate course in Modeling and Simulation of Physical Systems [FEMT16](https://nastava.fesb.unist.hr/nastava/predmeti/11623), taught by professors Dragan Poljak and Ivica Puljak. 

In [1]:
# libs
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.dates as mdates
import datetime as dt

from covid_19.utils import normalize, restore, moving_average, train_test_split
from covid_19.plotting import plotData, figsize, latexconfig
from covid_19.exponential_models import ExponentialModel, LogisticModel
from covid_19.compartmental_models import SIR, SEIR

# enable LaTex figure formatting
latexconfig()

In [2]:
# data
confirmed_cases = np.loadtxt('data/confirmed_cases.dat')
recovered_cases = np.loadtxt('data/recovered_cases.dat')

# train-test split 
ratio = 1.
train_confirmed_cases, test_confirmed_cases = train_test_split(confirmed_cases, ratio)
train_recovered_cases, test_recovered_cases = train_test_split(recovered_cases, ratio)

# days since first case
x = np.arange(len(confirmed_cases))

In [3]:
S0, E0, I0, R0 = 4e6, 4, 2, 0 # rough estimate, from GP fit upper bound
initial_conditions = [S0, E0, I0, R0]
N = np.sum(initial_conditions)
n_future_days = len(confirmed_cases) * 5

In [4]:
seir_model = SEIR()

In [5]:
beta, delta, alpha, gamma = seir_model.fit(confirmed_cases, recovered_cases, initial_conditions)

In [9]:
sol = seir_model.predict(n_future_days)